In [1]:
import duckdb
import pyarrow.dataset as ds
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from models.eta_model_trainer import ETAModelTrainer

2023-02-21 22:33:50,027 - youconfigme.youconfigme - INFO - searching for config on /home/dml/proyectos/bicisba/research/models/settings.ini
2023-02-21 22:33:50,028 - youconfigme.youconfigme - INFO - searching for config on /home/dml/proyectos/bicisba/research/settings.ini


In [2]:
dataset = ds.dataset("data/status", format="parquet", partitioning="hive")

In [3]:
con = duckdb.connect()
con = con.register("status", dataset)

In [27]:
station_id = 3
df_query = f"""
WITH base_status AS (select
    station_id,
    hour,
    num_bikes_available,
    num_bikes_disabled,
    num_docks_available,
    num_docks_disabled,
    status,
    make_timestamp(year, month, day, hour, minute, 0.0) as ts,
from
    status
where
    year = 2022 and
    month >= 1 and
    month <= 4 and
    station_id = {station_id} and
    status = 'IN_SERVICE'),
status_by_minute AS ("""
df_query += " union ".join([
f"""
select
    station_id,
    hour,
    dayofweek(ts) as dow,
    num_bikes_available,
    num_bikes_disabled,
    num_docks_available,
    num_docks_disabled,
    minute(lead(ts, {i}) over (
        order by ts asc
    ) - ts)  as minutes_bt_check,
    lead(num_bikes_available, {i}) over (
        order by ts asc
    ) as bikes_available,
from
    base_status
""" for i in range(1, 16)])
df_query += """)
SELECT station_id,
    hour,
    dow,
    num_bikes_disabled,
    num_docks_available,
    num_docks_disabled,
    minutes_bt_check
FROM status_by_minute
WHERE (num_bikes_available = 0
AND bikes_available > 0)
OR (bikes_available > num_bikes_available
AND num_bikes_available <= 3)
"""
dataset_df = con.execute(df_query).df()
len(dataset_df)

19704

In [28]:
dataset_df.head()

,station_id,hour,dow,num_bikes_disabled,num_docks_available,num_docks_disabled,minutes_bt_check
0,3,22,2,0,20,0,22
1,3,19,0,0,19,0,19
2,3,11,1,0,18,0,24
3,3,11,3,0,19,0,25
4,3,13,3,0,18,0,17


In [18]:
FEATURES_ORDER = ["hour", "dow", "num_bikes_disabled", "num_docks_available", "num_docks_disabled"]
rf_pipeline = make_pipeline(ColumnTransformer([("ohe",  OneHotEncoder(sparse=False), [0, 1]), ("ss",  StandardScaler(), slice(2,5))]),
                            MLPRegressor((128, 128, 128)))

In [19]:
X_train, X_test, y_train, y_test = train_test_split(dataset_df[FEATURES_ORDER], dataset_df["minutes_bt_check"], test_size=0.2, shuffle=False)

In [20]:
rf_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  [0, 1]),
                                                 ('ss', StandardScaler(),
                                                  slice(2, 5, None))])),
                ('mlpregressor',
                 MLPRegressor(hidden_layer_sizes=(128, 128, 128)))])

In [21]:
mean_absolute_error(y_test, rf_pipeline.predict(X_test))

5.269380577081491

In [59]:
from models.s3 import S3Client
from io import BytesIO 
import joblib 

s3_cli = S3Client()

In [60]:
with BytesIO() as mem_f:
    joblib.dump(rf_pipeline, mem_f)
    mem_f.seek(0)
    s3_cli.client.Bucket("frame").upload_fileobj(Key="models/current_eta_model.joblib", Fileobj=mem_f) 


/home/dml/.cache/pypoetry/virtualenvs/research-bicisba-A0HrnKHR-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minio.crossnox.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [61]:
with BytesIO() as mem_f:
    s3_cli.client.Bucket("frame").download_fileobj(Key="models/current_eta_model.joblib", Fileobj=mem_f)
    mem_f.seek(0)
    loaded_model = joblib.load(mem_f)

loaded_model.predict(X_test[:3])

/home/dml/.cache/pypoetry/virtualenvs/research-bicisba-A0HrnKHR-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minio.crossnox.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/dml/.cache/pypoetry/virtualenvs/research-bicisba-A0HrnKHR-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minio.crossnox.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


array([ 9.01762462, 14.37548957,  8.96427488])

# Whole pipeline

In [2]:
CURRENT_DATE = "2022/09/01"
avail_model_trainer = ETAModelTrainer()
dataset_df = avail_model_trainer.create_dataset(CURRENT_DATE)

In [3]:
avail_model_trainer.train_all_stations(dataset_df)

In [ ]:
DATES = pd.date_range(end=CURRENT_DATE, periods=3)
avail_model_trainer.dump_stations_pipelines(DATES[-1], current=True)

In [5]:
len(dataset_df)/len(dataset_df.station_id.unique())

3572.75